In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import sys
import matplotlib.pyplot as plt
import random
import time
#from nxsim import BaseNetworkAgent
from datetime import datetime
import collections
from collections import defaultdict
import copy
import math
nx.__version__

'2.2'

In [2]:
# Set some parameters to apply to all plots. These can be overridden
# in each plot if desired
import matplotlib
# Plot size to 14" x 7"
matplotlib.rc('figure', figsize = (24, 12))
# Font size to 14
matplotlib.rc('font', size = 20)
# Add grid lines
matplotlib.rc('axes', grid = False)
# Set backgound color to white
matplotlib.rc('axes', facecolor = 'white')

In [142]:
read_data = pd.read_csv("pnas.1713568115.sd01.txt") #read the data file, no cleaning required
read_data = read_data[read_data['trial'] == 1]  #take the data for the 1st trial only
data = read_data
data.head(10)
#print(len(data))
#type(data)



data['duration'] = data['end'] - data['begin']#store the diff btw begin and end to get total time for each trans. in ms
data['duration'] = data['duration']/1000.0 #change 'ms' to 's'
#data['time'] = data['time'].map(lambda x: [x]) #convert the time attr to list as as there are multiple iteractions between two bees
data.head()

,trial,id1,id2,begin,end,duration
0,1,201,219,1.373105e+12,1.373105e+12,2.000
1,1,201,219,1.373109e+12,1.373109e+12,42.000
2,1,201,242,1.373119e+12,1.373119e+12,2.000
3,1,201,242,1.373119e+12,1.373119e+12,2.999
4,1,201,255,1.372958e+12,1.372958e+12,41.999


In [143]:
data['begin'] = data['begin']/1000.0 #change 'ms' to 's'
data['day'] = data['begin'].map(lambda x: int(datetime.utcfromtimestamp(x).strftime('%d')))
data.head(10)

,trial,id1,id2,begin,end,duration,day
0,1,201,219,1.373105e+09,1.373105e+12,2.000,6
1,1,201,219,1.373109e+09,1.373109e+12,42.000,6
2,1,201,242,1.373119e+09,1.373119e+12,2.000,6
3,1,201,242,1.373119e+09,1.373119e+12,2.999,6
4,1,201,255,1.372958e+09,1.372958e+12,41.999,4
5,1,201,259,1.372937e+09,1.372937e+12,15.000,4
6,1,201,259,1.373041e+09,1.373041e+12,3.000,5
7,1,201,259,1.373240e+09,1.373240e+12,8.000,7
8,1,201,262,1.373360e+09,1.373360e+12,27.999,9
9,1,201,263,1.373178e+09,1.373178e+12,2.000,7


In [144]:
data['UTC'] = data['begin'].map(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
data['hours'] = data['begin'].map(lambda x: int(datetime.utcfromtimestamp(x).strftime('%H')))
data = data.sort_values('begin')
data.head(20)

,trial,id1,id2,begin,end,duration,day,UTC,hours
102932,1,612,1697,1.372829e+09,1.372829e+12,3.000,3,2013-07-03 05:29:18,5
138807,1,740,829,1.372829e+09,1.372829e+12,2.000,3,2013-07-03 05:29:23,5
225447,1,1138,1305,1.372829e+09,1.372829e+12,5.999,3,2013-07-03 05:29:32,5
11636,1,267,876,1.372829e+09,1.372829e+12,11.000,3,2013-07-03 05:29:41,5
126121,1,701,912,1.372829e+09,1.372829e+12,10.000,3,2013-07-03 05:29:41,5
119604,1,667,1460,1.372829e+09,1.372829e+12,17.000,3,2013-07-03 05:29:49,5
189002,1,938,1322,1.372829e+09,1.372829e+12,2.000,3,2013-07-03 05:29:52,5
243670,1,1248,1657,1.372829e+09,1.372829e+12,4.000,3,2013-07-03 05:30:18,5
234946,1,1205,1593,1.372829e+09,1.372829e+12,16.999,3,2013-07-03 05:30:25,5
21359,1,288,643,1.372829e+09,1.372829e+12,2.000,3,2013-07-03 05:30:25,5


In [145]:
data_day_3 = data[data['day'] == 3]
len(data_day_3)

30889

In [146]:
data_day_3['duration'] = data_day_3['duration'].map(lambda x: [x]) #convert the begin attr to list as as there are multiple iteractions between two bees
data_day_3['begin'] = data_day_3['begin'].map(lambda x: [x]) #convert the begin attr to list as as there are multiple iteractions between two bees

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [147]:
data_day_3.head(20)

,trial,id1,id2,begin,end,duration,day,UTC,hours
102932,1,612,1697,[1372829358.609],1.372829e+12,[3.0],3,2013-07-03 05:29:18,5
138807,1,740,829,[1372829363.609],1.372829e+12,[2.0],3,2013-07-03 05:29:23,5
225447,1,1138,1305,[1372829372.609],1.372829e+12,[5.999],3,2013-07-03 05:29:32,5
11636,1,267,876,[1372829381.608],1.372829e+12,[11.0],3,2013-07-03 05:29:41,5
126121,1,701,912,[1372829381.608],1.372829e+12,[10.0],3,2013-07-03 05:29:41,5
119604,1,667,1460,[1372829389.608],1.372829e+12,[17.0],3,2013-07-03 05:29:49,5
189002,1,938,1322,[1372829392.608],1.372829e+12,[2.0],3,2013-07-03 05:29:52,5
243670,1,1248,1657,[1372829418.608],1.372829e+12,[4.0],3,2013-07-03 05:30:18,5
234946,1,1205,1593,[1372829425.608],1.372829e+12,[16.999],3,2013-07-03 05:30:25,5
21359,1,288,643,[1372829425.608],1.372829e+12,[2.0],3,2013-07-03 05:30:25,5


In [148]:
data_day_3 = data_day_3.groupby(['id1','id2']).agg({'begin': 'sum', 'duration':'sum'})

In [149]:
data_day_3.head(10)

begin       duration
id1 id2                                                 
201 268                  [1372872921.921]          [5.0]
    272  [1372847419.324, 1372847462.323]  [26.999, 2.0]
    351                   [1372888861.67]          [2.0]
    357                  [1372889999.652]          [2.0]
    363  [1372831883.569, 1372831895.569]     [3.0, 2.0]
    376                  [1372841882.411]          [2.0]
    380                  [1372895570.564]          [2.0]
    393                  [1372865316.041]          [3.0]
    439                  [1372883191.759]          [2.0]
    444                  [1372839146.454]          [3.0]

In [150]:
data_day_3 = data_day_3.reset_index() #reset index after group by as there are some none values for column id1

In [151]:
data_day_3.head(200)

,id1,id2,begin,duration
0,201,268,[1372872921.921],[5.0]
1,201,272,"[1372847419.324, 1372847462.323]","[26.999, 2.0]"
2,201,351,[1372888861.67],[2.0]
3,201,357,[1372889999.652],[2.0]
4,201,363,"[1372831883.569, 1372831895.569]","[3.0, 2.0]"
5,201,376,[1372841882.411],[2.0]
6,201,380,[1372895570.564],[2.0]
7,201,393,[1372865316.041],[3.0]
8,201,439,[1372883191.759],[2.0]
9,201,444,[1372839146.454],[3.0]


#### Infinite tau value - add all the durations

In [152]:
data_day_3['sum_duration'] = data_day_3['duration'].map(lambda x: sum(x))

In [153]:
data_day_3.head(200)

,id1,id2,begin,duration,sum_duration
0,201,268,[1372872921.921],[5.0],5.000
1,201,272,"[1372847419.324, 1372847462.323]","[26.999, 2.0]",28.999
2,201,351,[1372888861.67],[2.0],2.000
3,201,357,[1372889999.652],[2.0],2.000
4,201,363,"[1372831883.569, 1372831895.569]","[3.0, 2.0]",5.000
5,201,376,[1372841882.411],[2.0],2.000
6,201,380,[1372895570.564],[2.0],2.000
7,201,393,[1372865316.041],[3.0],3.000
8,201,439,[1372883191.759],[2.0],2.000
9,201,444,[1372839146.454],[3.0],3.000


In [154]:
len(data_day_3)

24933

In [160]:
tau_values = [30,60,90,120] #in seconds
for index,row in data_day_3.iterrows():
    if len(row[2]) >1:
        index_list = []
        any_name = []
        for i in range(0,len(row[2]) -1 ): #begin column
            time_diff = row[2][i+1] - row[2][i]
            if time_diff <= tau_values[3]:
                #index_list.append(i)
                #print(index)
                #print("i=",i)
                any_name.append(row[3][i] + row[3][i+1])
                #print(row[2])
                #print("time dur = ", row[3])
                #row[3][i+1] = row[3][i] + row[3][i+1] 
                #row[3].pop(i)
                #row[2].pop(i)
            else:
                any_name.append(row[3][i])
        print(row[3],any_name)
       
        #data_day_3.loc[index,'duration'] = any_name
        #row[3] = any_name
        

[26.999, 2.0] [28.999]
[3.0, 2.0] [5.0]
[3.0, 2.0] [5.0]
[42.0, 11.999, 18.999] [53.999, 30.997999999999998]


ValueError: Must have equal len keys and value when setting with an iterable

In [159]:
data_day_3.head(200)

,id1,id2,begin,duration,sum_duration
0,201,268,[1372872921.921],[5.0],5.000
1,201,272,"[1372847419.324, 1372847462.323]","[26.999, 2.0]",28.999
2,201,351,[1372888861.67],[2.0],2.000
3,201,357,[1372889999.652],[2.0],2.000
4,201,363,"[1372831883.569, 1372831895.569]","[3.0, 2.0]",5.000
5,201,376,[1372841882.411],[2.0],2.000
6,201,380,[1372895570.564],[2.0],2.000
7,201,393,[1372865316.041],[3.0],3.000
8,201,439,[1372883191.759],[2.0],2.000
9,201,444,[1372839146.454],[3.0],3.000


In [157]:
for index,row in data_day_3.iterrows():
    if len(row[2]) != len(row[3]):
        print("Beeeeeeee")